# MMG thrust model

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,8)
from src.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from src.models.MMG_propeller import *
from src.substitute_dynamic_symbols import run
from src.prime_system import PrimeSystem
from scipy.optimize import least_squares

In [ ]:
ship="kvlcc2_hsva"
vmm_name = "vmm_martins_simple"
#vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)

ship_data = catalog.load(f"{ship}.ship_data")

train = "HSVA_CPMC_KVLCC2_Z_25_05"
df_train = catalog.load(f'{ship}.updated.{train}.data_ek_smooth')
df_train_raw = catalog.load(f'{ship}.{train}.raw_data')


test = "HSVA_CPMC_KVLCC2_Z_35_05"
df_test = catalog.load(f'{ship}.updated.{test}.data_ek_smooth')

In [ ]:
X_P_solution[0][X_P]

In [ ]:
lambda_X_P = sp.lambdify(list(X_P_solution[0][X_P].free_symbols), X_P_solution[0][X_P])


In [ ]:
lambda_X_P

In [ ]:
df_ = df_train.copy()
df_['rev'] = df_train_raw['rev'].values
#J_ = open_water_characteristics.index
#n_ = 1.0
D_ = ship_data['D']
w0_ = ship_data['w_p0']
rho_ = 1000
tdf_ = ship_data['tdf']
#df_['J'] = J_
#df_['u'] = J_*n_*D_/(1-w0_)
#df_['v'] = 0
#df_['r'] = 0
#df_['rev'] = n_
df_['U'] = np.sqrt(df_['u']**2 + df_['v']**2)

df_['beta'] = -np.arctan2(df_['v'], df_['u'])
ps = PrimeSystem(**ship_data)
xp_prime = ps._prime(
    ship_data["x_p"],
    unit="length",
)
r_prime = ps._prime(df_['r'], unit="angular_velocity", U=df_["U"])
df_['beta_p'] = df_['beta'] - xp_prime * r_prime


df_['C_2'] = np.where(df_['beta_p'] > 0, ship_data["C_2_beta_p_pos"], ship_data["C_2_beta_p_neg"])
#df_.set_index('J', inplace=True)

In [ ]:
propeller_coefficients = catalog.load("kvlcc2.propeller_coefficients")

In [ ]:
df_['fx'] = run(function=lambda_X_P, inputs=df_, **ship_data, **propeller_coefficients)

In [ ]:
df_

In [ ]:
df_['thrust_prediction'] = df_['fx']/(1-ship_data['tdf'])

In [ ]:
df_.plot(y=['thrust','thrust_prediction'])

In [ ]:
X_P_solution[0][X_P]

In [ ]:
lambda_X_P

In [ ]:
fig,ax=plt.subplots()
mask = df_['beta_p'] > 0
df_.loc[mask].plot(y='thrust', style='.', label=r'$\beta_p>0$', ax=ax)
df_.loc[~mask].plot(y='thrust', style='.', label=r'$\beta_p<0$', ax=ax)

In [ ]:
df_

In [ ]:
df_input = df_.drop(columns=['C_2'])
mask = df_['beta_p'] > 0
df_input = df_input.loc[mask].copy()
ship_data_ = ship_data.copy()
ship_data_.pop('C_1')
ship_data_.pop('tdf')

In [ ]:
def fun(x):
    
    C_1 = x[0]
    C_2 = x[1]
    tdf = x[2]
    
    fx = run(function=lambda_X_P, inputs=df_input, **ship_data_, **propeller_coefficients, C_1=C_1, C_2=C_2, tdf=tdf)
    
    thrust_prediction = fx/(1-tdf)
    
    epsilon = np.array(df_input['thrust'] - thrust_prediction)
    
    return epsilon

In [ ]:
x = df_[['C_2']].copy()

In [ ]:
fig,ax=plt.subplots()
ax.plot(fun([2,1.6,ship_data['tdf']]),'.')
ax.plot(fun([2,0.6,ship_data['tdf']]),'.')
ax.plot(fun([2,2.6,ship_data['tdf']]),'.')

In [ ]:
result = least_squares(fun=fun, x0=(2,1.6,ship_data['tdf']), method='dogbox')

In [ ]:
result

In [ ]:
df2 = df_input.copy()
df2['C_1'] = result.x[0]
df2['C_2'] = result.x[1]
tdf = result.x[2]

In [ ]:
df2['fx'] = run(function=lambda_X_P, inputs=df2, **ship_data_, **propeller_coefficients, tdf=tdf)
df2['thrust_prediction'] = df2['fx']/(1-tdf)

In [ ]:
fig,ax=plt.subplots()
df2.loc[mask].plot(y=['thrust','thrust_prediction'], style='.', ax=ax)



In [ ]:
df2 = df_input.copy()
df2['C_1'] = 2
df2['C_2'] = 2

df2['fx'] = run(function=lambda_X_P, inputs=df2, **ship_data_, **propeller_coefficients, tdf=0.30)
df2['thrust_prediction'] = df2['fx']/(1-0.30)

fig,ax=plt.subplots()
df2.loc[mask].plot(y=['thrust','thrust_prediction'], style='.', ax=ax)


In [ ]:
eqs = [
    eq_T,
    eq_J,
    eq_K_T,
]

solution = sp.solve(eqs, thrust, K_T, J, dict=True)[0][thrust]
eq_thrust_simple = sp.Eq(thrust, solution)

In [ ]:
eq_thrust_simple

In [ ]:
lambda_thrust_simple = sp.lambdify(list(eq_thrust_simple.rhs.free_symbols), eq_thrust_simple.rhs)

In [ ]:
df_result = df_.copy()
df_result['thrust'] = run(lambda_thrust_simple, inputs=df_, **propeller_coefficients, **ship_data, w_p=ship_data['w_p0']) 

In [ ]:
fig,ax=plt.subplots()
df_.plot(y='thrust', ax=ax)
df_result.plot(y='thrust', ax=ax)

v_p = df_['v']+df_['r']*ship_data['x_r']

fig,ax=plt.subplots()
v_p.plot(ax=ax)
df_.plot(y='delta', ax=ax)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linear_regression = LinearRegression()

In [ ]:
X = pd.DataFrame(index=df_.index)
X['delta'] = df_['delta']
X['delta**2'] = df_['delta']**2
X['v'] = df_['v']
X['r'] = df_['r']
X['v_p'] = v_p
X['v_p**2'] = v_p**2


y = df_['thrust'] - df_result['thrust']
linear_regression.fit(X=X,y=y)

In [ ]:
df_result['thrust_predict'] = df_result['thrust'] + linear_regression.predict(X)

In [ ]:
fig,ax=plt.subplots()
df_.plot(y='thrust', ax=ax)
df_result.plot(y=['thrust', 'thrust_predict'], ax=ax)

v_p = df_['v']+df_['r']*ship_data['x_r']

fig,ax=plt.subplots()
v_p.plot(ax=ax)
df_.plot(y='delta', ax=ax)

In [ ]:
eqs = [
    eq_T,
    eq_J,
    eq_K_T,
]

solution = sp.solve(eq_thrust_simple, w_p,  dict=True)[1][w_p]
eq_w_p = sp.Eq(w_p, solution)

In [ ]:
eq_w_p

In [ ]:
lambda_w_p = sp.lambdify(list(eq_w_p.rhs.free_symbols), eq_w_p.rhs)

In [ ]:
lambda_w_p

In [ ]:
df_result['w_p'] = run(lambda_w_p, inputs=df_, **ship_data, **propeller_coefficients)

In [ ]:
X = pd.DataFrame(index=df_.index)
X['delta'] = df_['delta']
#X['delta**2'] = df_['delta']**2
#X['v'] = df_['v']
#X['r'] = df_['r']
X['v_p'] = df_['v']+df_['r']*ship_data['x_r']
X['v_p**2'] = v_p**2


y = df_result['w_p']
linear_regression.fit(X=X,y=y)
df_result['w_p_predict'] = linear_regression.predict(X=X)

In [ ]:
fig,ax=plt.subplots()
df_result.plot(y=['w_p','w_p_predict'],ax=ax)
ax2 = ax.twinx()
df_result.plot(y=['delta'],ax=ax2)


ax3 = ax.twinx()
X.plot(y=['v_p'],ax=ax3)

In [ ]:
df_result['thrust'] = run(lambda_thrust_simple, inputs=df_, **propeller_coefficients, **ship_data_, w_p=df_result['w_p_predict']) 

In [ ]:
fig,ax=plt.subplots()
df_.plot(y='thrust', ax=ax)
df_result.plot(y='thrust', ax=ax)

In [ ]:
linear_regression_pos = LinearRegression()
mask = df_result['beta_p'] > 0
df_pos = df_result.loc[mask].copy()
X = pd.DataFrame(index=df_pos.index)
X['delta'] = df_pos['delta']
X['v_p'] = df_pos['v'] + df_pos['r']*ship_data['x_p']
X['v_p**2'] = X['v_p']**2
y = df_pos['w_p']
linear_regression_pos.fit(X=X,y=y)
df_pos['w_p_predict'] = linear_regression_pos.predict(X=X)

In [ ]:
linear_regression_neg = LinearRegression()
mask = df_result['beta_p'] <= 0
df_neg = df_result.loc[mask].copy()
X = pd.DataFrame(index=df_neg.index)
X['delta'] = df_neg['delta']
X['v_p'] = df_neg['v'] + df_neg['r']*ship_data['x_p']
X['v_p**2'] = X['v_p']**2
y = df_neg['w_p']
linear_regression_neg.fit(X=X,y=y)
df_neg['w_p_predict'] = linear_regression_neg.predict(X=X)

In [ ]:
fig,ax=plt.subplots()
df_pos.plot(y=['w_p','w_p_predict'], style='.',ax=ax)
df_neg.plot(y=['w_p','w_p_predict'], style='.',ax=ax)

In [ ]:
df_result = df_.copy()
X = pd.DataFrame(index=df_result.index)
X['delta'] = df_result['delta']
X['v_p'] = df_result['v'] + df_result['r']*ship_data['x_p']
X['v_p**2'] = X['v_p']**2

df_result['w_p'] = np.where(df_result['beta_p'] > 0,  linear_regression_pos.predict(X), linear_regression_neg.predict(X))
df_result['thrust'] = run(lambda_thrust_simple, inputs=df_result, **propeller_coefficients, **ship_data_) 

In [ ]:
fig,ax=plt.subplots()
df_.plot(y='thrust', label='True', ax=ax)
df_result.plot(y='thrust', label='Prediction', ax=ax)


In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_true=df_['thrust'], y_pred=df_result['thrust'])

In [ ]:
model = catalog.load(f"{ship}.updated.{vmm_name}.joined.model")
result = model.simulate(df_result)

In [ ]:
dataframes = {
    'Experiment':df_result,
    'Sim predicted thrust':result.result
}
track_plots(dataframes=dataframes, lpp=ship_data['L'], beam=ship_data['B'])

In [ ]:
plot(dataframes=dataframes,keys=['psi','y0','u','v','r'], ncols=1);

## All tests

In [ ]:
def preprocess(data):

    data['beta'] = -np.arctan2(data['v'], data['u'])
    ps = PrimeSystem(**ship_data)
    xp_prime = ps._prime(
        ship_data["x_p"],
        unit="length",
    )
    data['U'] = np.sqrt(data['u']**2 + data['v']**2)
    r_prime = ps._prime(data['r'], unit="angular_velocity", U=data["U"])
    data['beta_p'] = data['beta'] - xp_prime * r_prime
    data['w_p'] = run(lambda_w_p, inputs=data, **ship_data_, **propeller_coefficients)
    
    return data

In [ ]:
data = catalog.load(f"{ship}.updated.joined.data_ek_smooth")
data = preprocess(data)

In [ ]:
data.plot(y=['w_p'])

In [ ]:
def features(df, add_constant=False):
    
    X = pd.DataFrame(index=df.index.copy())
    X['delta'] = df['delta']
    X['delta**2'] = df['delta']**2
    v_p = df['v'] + df['r']*ship_data['x_p']
    X['v_p'] = v_p
    X['v_p**2'] = v_p**2
    
    if add_constant:
        X = sm.tools.add_constant(X)
    
    return X
    

In [ ]:
import statsmodels.api as sm

In [ ]:
def fit(data, add_constant=False):
    
    data = preprocess(data)
    
    mask = data['beta_p'] > 0
    df_pos = data.loc[mask].copy()
    model = {}
    X = features(df_pos)
    y = df_pos['w_p'] - ship_data['w_p0']
    linear_regression_pos = LinearRegression(fit_intercept=add_constant)
    model['pos'] = linear_regression_pos.fit(X=X,y=y)
    
    mask = data['beta_p'] <= 0
    df_neg = data.loc[mask].copy()
    
    X = features(df_neg)
    y = df_neg['w_p'] - ship_data['w_p0']
    linear_regression_neg = LinearRegression(fit_intercept=add_constant)
    model['neg'] = linear_regression_neg.fit(X=X,y=y)
    
    
    return model
    
def fit_sm(data, add_constant=False):
    
    data = preprocess(data)
    
    mask = data['beta_p'] > 0
    df_pos = data.loc[mask].copy()
    model = {}
    X = features(df_pos, add_constant=add_constant)
    y = df_pos['w_p'] - ship_data['w_p0']
    linear_regression_pos = sm.OLS(y,X,hasconst=add_constant)
    model['pos'] = linear_regression_pos.fit()
    
    mask = data['beta_p'] <= 0
    df_neg = data.loc[mask].copy()
    
    X = features(df_neg, add_constant=add_constant)
    y = df_neg['w_p'] - ship_data['w_p0']
    linear_regression_neg = sm.OLS(y,X,hasconst=add_constant)
    model['neg'] = linear_regression_neg.fit()
    
    
    return model    

def predict(model, data, propeller_coefficients, ship_data, add_constant=False):
    
    X = features(data, add_constant=add_constant)
    df_result = data.copy()
    df_result['w_p'] = np.where(data['beta_p'] > 0,  model['pos'].predict(X), model['neg'].predict(X)) + ship_data['w_p0']
    df_result['thrust'] = run(lambda_thrust_simple, inputs=df_result, **propeller_coefficients, **ship_data) 
    return df_result

In [ ]:
model_thrust = fit(data, add_constant=add_constant)
add_constant=False
model_thrust_sm = fit_sm(data, add_constant=add_constant)

df_result = predict(model=model_thrust, data=data, propeller_coefficients=propeller_coefficients, ship_data=ship_data, add_constant=add_constant)

In [ ]:
model_thrust_sm['pos'].summary()

In [ ]:
model_thrust['pos'].coef_

In [ ]:
model_thrust_sm['neg'].summary()

In [ ]:
model_thrust['neg'].coef_

In [ ]:
l = model_thrust['neg']
l.intercept_

In [ ]:
r2_score(y_true=data['thrust'], y_pred=df_result['thrust'])

In [ ]:
fig,ax=plt.subplots()
data.plot(y='w_p', ax=ax)
df_result.plot(y='w_p', ax=ax)

In [ ]:
fig,ax=plt.subplots()
data.plot(y='thrust', ax=ax)
df_result.plot(y='thrust', ax=ax)

In [ ]:
#df_test = data.sample(n=int(len(data)*0.2), replace=False, random_state=42)
df_test = data.iloc[-int(len(data)*0.2):-1]


index_train = list(set(data.index) - set(df_test.index))
df_train = data.loc[index_train].copy()

fig,ax=plt.subplots()
df_train.plot(y='thrust', style='.', ax=ax)
df_test.plot(y='thrust', style='.', ax=ax)


In [ ]:
len(df_train)

In [ ]:
len(df_test)

In [ ]:
len(data)

In [ ]:
thrust_model = fit_sm(data=df_train)

In [ ]:
df_result = predict(model=thrust_model, data=df_test, propeller_coefficients=propeller_coefficients, ship_data=ship_data_)

In [ ]:
r2_score(y_true=df_test['thrust'], y_pred=df_result['thrust'])

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
time_windows = TimeSeriesSplit(n_splits=10)

r2s = []
for train_index, test_index in time_windows.split(data):
    
    df_train = data.loc[train_index]
    df_test = data.loc[test_index]
    
    thrust_model = fit_sm(data=df_train)
    df_result = predict(model=thrust_model, data=df_test, propeller_coefficients=propeller_coefficients, ship_data=ship_data_)
    r2 = r2_score(y_true=df_test['thrust'], y_pred=df_result['thrust'])
    
    r2s.append(r2)


In [ ]:
r2s